In [1]:
import requests as http
from io import StringIO
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from av_lib import *

In [2]:
api_key = None
with open('api-key.txt') as file:
    api_key = file.read()

In [3]:
symbol = Symbol('AAPL', 'Apple Inc.', 'USA')

In [4]:
url = daily_adjusted_query(api_key, symbol=symbol.tick(), output_size='full')
data = fetch_data(url)

In [5]:
data = StringIO(data)
data = pd.read_csv(data)
data

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-09,142.340,145.5700,140.90,142.16,142.160000,76097011,0.0,1.0
1,2022-12-08,142.360,143.5200,141.10,142.65,142.650000,62128338,0.0,1.0
2,2022-12-07,142.190,143.3700,140.00,140.94,140.940000,69721094,0.0,1.0
3,2022-12-06,147.075,147.3000,141.92,142.91,142.910000,64727186,0.0,1.0
4,2022-12-05,147.770,150.9199,145.77,146.63,146.630000,68826442,0.0,1.0
...,...,...,...,...,...,...,...,...,...
5811,1999-11-05,84.620,88.3700,84.00,88.31,0.672034,3721500,0.0,1.0
5812,1999-11-04,82.060,85.3700,80.62,83.62,0.636344,3384700,0.0,1.0
5813,1999-11-03,81.620,83.2500,81.00,81.50,0.620210,2932700,0.0,1.0
5814,1999-11-02,78.000,81.6900,77.31,80.25,0.610698,3564600,0.0,1.0


In [6]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-12-09,142.340,145.5700,140.90,142.16,142.16,76097011,0.0,1.0
1,2022-12-08,142.360,143.5200,141.10,142.65,142.65,62128338,0.0,1.0
2,2022-12-07,142.190,143.3700,140.00,140.94,140.94,69721094,0.0,1.0
3,2022-12-06,147.075,147.3000,141.92,142.91,142.91,64727186,0.0,1.0
4,2022-12-05,147.770,150.9199,145.77,146.63,146.63,68826442,0.0,1.0


In [7]:
adj_close = px.line(
    data_frame=data, 
    x='timestamp', 
    y='adjusted_close', 
    title=f'Adjusted Stock Price - {symbol.tick()}',
    labels={
        'timestamp': 'Date',
        'adjusted_close': 'Price'
    },
    log_y=True
)
adj_close.show()

In [8]:
data['timedelta'] = (data['timestamp'] - datetime.today()).dt.days
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,timedelta
0,2022-12-09,142.340,145.5700,140.90,142.16,142.16,76097011,0.0,1.0,-4
1,2022-12-08,142.360,143.5200,141.10,142.65,142.65,62128338,0.0,1.0,-5
2,2022-12-07,142.190,143.3700,140.00,140.94,140.94,69721094,0.0,1.0,-6
3,2022-12-06,147.075,147.3000,141.92,142.91,142.91,64727186,0.0,1.0,-7
4,2022-12-05,147.770,150.9199,145.77,146.63,146.63,68826442,0.0,1.0,-8


In [9]:
adj_close = px.line(
    data_frame=data, 
    x='timedelta', 
    y='adjusted_close', 
    title=f'Adjusted Stock Price - {symbol.tick()}',
    labels={
        'timedelta': 'Days Before',
        'adjusted_close': 'Price'
    },
    log_y=True
)
adj_close.show()

In [10]:
import statsmodels.formula.api as smf

model = smf.ols(data=data, formula='np.log(adjusted_close) ~ timedelta').fit()
model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     np.log(adjusted_close)   R-squared:                       0.947
Model:                                OLS   Adj. R-squared:                  0.947
Method:                     Least Squares   F-statistic:                 1.043e+05
Date:                    Mon, 12 Dec 2022   Prob (F-statistic):               0.00
Time:                            14:24:57   Log-Likelihood:                -3668.8
No. Observations:                    5816   AIC:                             7342.
Df Residuals:                        5814   BIC:                             7355.
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3004      0.012    444.400      0.000       5.277       5.324
timedelta      0.0008   2.45e-06    322.910      0.000       0.001       0.001
==============================================================================
Omnibus:                       34.679   Durbin-Watson:                   0.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.257
Skew:                          -0.129   Prob(JB):                     1.81e-09
Kurtosis:                       3.315   Cond. No.                     9.75e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.75e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
next_5y = np.arange(360 * 5)
projection = np.exp(model.predict(next_5y))
ax = sns.lineplot(
    x = pd.concat(data['timedelta'], next_5y), 
    y = pd.concat(data['adjusted_close'], projection)
)
ax  #sns.lineplot(ax=ax, x=next_5y, y = np.exp(model.predict(next_5y)))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/pedro/.local/lib/python3.10/site-packages/patsy/compat.py", line 36, in call_and_wrap_exc
    return f(*args, **kwargs)
  File "/home/pedro/.local/lib/python3.10/site-packages/patsy/eval.py", line 169, in eval
    return eval(code, {}, VarLookupDict([inner_namespace]
  File "<string>", line 1, in <module>
  File "/home/pedro/.local/lib/python3.10/site-packages/patsy/eval.py", line 52, in __getitem__
    return d[key]
  File "/home/pedro/.local/lib/python3.10/site-packages/patsy/eval.py", line 52, in __getitem__
    return d[key]
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/pedro/.local/lib/python3.10/site-packages/statsmodels/base/model.py", line 1137, in predict
    exog = dmatrix(design_info, exog, return_type="dataframe")
  File "/ho